# Set Eta Init -- spatially varying initial surface elevation

GeoClaw v5.6.0 has a single scalar parameter `sea_level` and the water surface is initialized to this value in every cell where the GeoClaw cell-averaged topography value `B` is smaller, i.e., the water depth in each cell is initialized to:

`h[i,j] = max(0, sea_level - B[i,j])`.

In some cases it is desirable to initialize the depth so that the surface is spatially varying.

Two examples:

First, in tsunami modeling of a nearfield event, the seafloor deformation due to an earthquake often extends onto shore in the region being modeled.  If the coastal region subsides, for example, then the land drops near the shore and the water adjacent drops as well.  If a grid patch was initialized before the deformation specified in the dtopo file by the formula above, then the depth `h[i,j]` does not decrease during the subsidence, which is the correct behavior.  However, in some cases the tsunami does not arrive at the shore quickly and so it is desirable to use coarser grids in early stages of the computation, introducing highly refined grids only after some specified time.  When new levels of refinement are first introduced into a simulation then the formula given above is used to initialize cells near the coast.  But if the coast subsided, the the formula should really be replaced by:

`h[i,j] = max(0, sea_level + dz[i,j] - B[i,j])`

where `dz[i,j]` is obtained by interpolating the co-seismic deformation specified in the dtopo file to the cell center.  Failure to do this can sometimes result in large areas being flooded by the initialization that should not be flooded by the tsunami.

Second, there may be onshore lakes whose initial surface elevation should be different than `sea_level`, or for modeling dam break problems there may be a collection of lakes of interest at different initial elevations.

Another situation where something special must be done is if an area should be dry even though the topography elevation is below `sea_level`.  This case is handled by a different set of tools described in [ForceDry.ipynb](ForceDry.ipynb), but they can also be combined with a spatially varying `eta_init`.


## The `set_eta_init` subroutine

A new subroutine has been introduced called `set_eta_init` that can be called for each grid patch and will return the desired `eta_init` value for each cell. A new boolean parameter `variable_eta_init` can be set to to `True` to indicate that this routine should be called.

The user can provide a custom routine.  For example, for a simulation where a lake behind a dam is desired to be set to one elevation while everywhere else there should be no water, this routine could check the `(x,y)` location of each cell and set `eta_init` either to the lake elevation or to a small value lower than any topography (to force `h = 0`) depending on the location.

A version of this routine called `set_eta_init_dtopo.f90` can be used to deal with the first situation described above, where we want to set `eta_init[i,j] = sea_level + dz[i,j]` in each cell, with `dz[i,j]` obtained by interpolating the co-seismic deformation specified in the dtopo file to the cell center. 

[Back to Index.ipynb](Index.ipynb), or go on to next notebook, [MarchingFront.ipynb](MarchingFront.ipynb).